In [2]:
%%capture
pip install mne

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import mne
import pywt

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%%capture
# folder = 'E:/Data WareHouse/1sec/Epileptic Seizure/equalized'
folder = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch'
epochs_path = [os.path.join(folder,i) for i in os.listdir(folder) if i[-3:]=='fif']

In [6]:
%%capture
# folder = 'E:/Data WareHouse/1sec/Epileptic Seizure/equalized'
# folder = '/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/equalized epoch'
# epochs_path = [os.path.join(folder,i) for i in os.listdir(folder) if i[-3:]=='fif']

data= [mne.read_epochs(i).pick_types(eeg=True) for i in epochs_path]
labels = [mne.read_epochs(i).events[:,2] for i in epochs_path]
group = [[i]*len(j) for i,j in enumerate(data)]
X=np.vstack(data)
Y=np.hstack(labels)
group= np.hstack(group)

In [7]:
print(X.shape,Y.shape,group.shape)

(5834, 29, 180) (5834,) (5834,)


In [8]:
np.savez('/content/drive/MyDrive/EEG Signal /Epileptic seizure/data/only data/raw_data.npz', X=X,Y=Y, Group = group)

# Feature Extraction methods

##Time-domain Features

1. Mean (mean_val): The average value of the signal amplitude, representing the signal's central tendency.

2. Median (median_val): The middle value of the signal amplitude when sorted, representing the central tendency of the signal, less sensitive to outliers than the mean.

3. Variance (var_val): Measures the spread of the signal's amplitude.

4. Standard Deviation (std_dev): The square root of the variance, representing the dispersion of the signal's amplitude.

5. Skewness (skewness): Measures the asymmetry of the signal's amplitude distribution around the mean.

6. Kurtosis (kurt): Measures the 'tailedness' of the signal's amplitude distribution, indicating the presence of outliers.

7. Zero Crossing Rate (zcr): The rate at which the signal changes sign, indicating frequency content.

8. Root Mean Square (rms_val): Represents the square root of the average of the squares of the signal, indicative of the signal's magnitude.

9. Signal Energy (energy): The sum of the squares of the signal values, indicative of the signal's power.

10. Crest Factor (crest_fact): The ratio of the peak amplitude of the waveform to the RMS value, indicating the extremeness of peaks.

11. Shape Factor (shape_fact): The ratio of the RMS value to the mean absolute value, indicative of the waveform shape.

12. Entropy (signal_entropy): Measures the unpredictability or complexity of the signal.

13. Peak Amplitude (peak_amp): The difference between the maximum and minimum amplitude, indicating the signal's range.

14. Number of Peaks (num_peaks): The count of local maxima, indicating the frequency of oscillations.

15. Average Peak-to-Peak Distance (peak_to_peak_distance): The average distance between consecutive peaks, related to the periodicity of the signal.

16. Hjorth Parameters:

  Activity: Indicates the signal power.

  Mobility: Indicates the mean frequency or the proportion of the standard deviation of the power spectrum.
  
  Complexity: Indicates the bandwidth of the signal or the change in frequency.





In [ ]:
# import numpy as np
# from scipy.stats import skew, kurtosis
# from scipy.signal import find_peaks

# def zero_crossing_rate(signal):
#     zero_crossings = np.where(np.diff(np.sign(signal)))[0]
#     return len(zero_crossings) / len(signal)

# def hjorth_parameters(signal):
#     diff_input = np.diff(signal)
#     diff_diff_input = np.diff(diff_input)

#     activity = np.var(signal)
#     mobility = np.sqrt(np.var(diff_input)/activity)
#     complexity = np.sqrt(np.var(diff_diff_input)/np.var(diff_input)) / mobility

#     return activity, mobility, complexity




# def extract_time_domain_features(epochs):
#     features = []

#     for epoch in epochs:
#         epoch_features = []

#         for channel_data in epoch:
#             # Flatten the channel data
#             flattened_data = channel_data.flatten()

#             # Basic Time-Domain Features
#             mean_val = np.mean(flattened_data)
#             median_val = np.median(flattened_data)
#             var_val = np.var(flattened_data)
#             std_dev = np.std(flattened_data)
#             skewness = skew(flattened_data)
#             kurt = kurtosis(flattened_data)
#             zcr = zero_crossing_rate(flattened_data)
#             peak_amp = np.ptp(flattened_data)

#             # Hjorth Parameters
#             activity, mobility, complexity = hjorth_parameters(flattened_data)

#             # Additional Features
#             num_waves = len(find_peaks(flattened_data)[0])
#             wave_duration = len(flattened_data) / num_waves if num_waves > 0 else 0

#             channel_features = [
#                 mean_val, median_val, var_val, std_dev, skewness, kurt, zcr, num_waves,
#                 wave_duration, peak_amp, activity, mobility, complexity
#             ]
#             epoch_features.append(channel_features)

#         features.append(epoch_features)

#     return np.array(features)

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis, entropy
from scipy.signal import find_peaks


def hjorth_parameters(signal):
    diff_input = np.diff(signal)
    diff_diff_input = np.diff(diff_input)

    activity = np.var(signal)
    mobility = np.sqrt(np.var(diff_input)/activity)
    complexity = np.sqrt(np.var(diff_diff_input)/np.var(diff_input)) / mobility

    return activity, mobility, complexity

# def zero_crossing_rate(signal):
#     # Enhanced ZCR to handle noise
#     # Setting a threshold (eps) to consider as zero (to avoid detecting false crossings due to noise)
#     eps = 0.01 * np.std(signal)
#     zero_crossings = np.where(np.diff(np.sign(signal)))[0]
#     close_to_zero = np.where(np.abs(signal) < eps)[0]
#     return (len(zero_crossings) + len(close_to_zero)) / len(signal)

def zero_crossing_rate(signal):
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    return len(zero_crossings) / len(signal)

def safe_divide(numerator, denominator, default=0.0):
    """Safely divide two numbers, returning a default value if the denominator is zero."""
    if denominator == 0:
        return default
    else:
        return numerator / denominator

def signal_entropy(signal):
    # Calculate histogram of the signal
    hist, bin_edges = np.histogram(signal, bins='auto', density=True)
    # Ensure non-zero histogram values by adding a small constant
    hist += np.finfo(float).eps
    # Normalize the histogram to get a probability distribution
    prob_dist = hist / hist.sum()
    # Calculate the entropy
    return entropy(prob_dist)

def rms(signal):
    return np.sqrt(np.mean(signal**2))

def signal_energy(signal):
    return np.sum(signal**2)



def crest_factor(signal, rms_val=None):
    if rms_val is None:
        rms_val = rms(signal)
    return np.max(np.abs(signal)) / rms_val

def shape_factor(signal, rms_val=None):
    if rms_val is None:
        rms_val = rms(signal)
    mean_abs = np.mean(np.abs(signal))
    return rms_val / mean_abs if mean_abs != 0 else 0

def extract_time_domain_features(epochs):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Flatten the channel data if needed
            flattened_data = np.ravel(channel_data)

            # Basic Time-Domain Features
            mean_val = np.mean(flattened_data)
            median_val = np.median(flattened_data)
            var_val = np.var(flattened_data)
            std_dev = np.std(flattened_data)
            skewness = skew(flattened_data)
            kurt = kurtosis(flattened_data)
            zcr = zero_crossing_rate(flattened_data)
            rms_val = rms(flattened_data)
            energy = signal_energy(flattened_data)
            crest_fact = crest_factor(flattened_data, rms_val)
            shape_fact = shape_factor(flattened_data, rms_val)
            signal_entropy_val = signal_entropy(flattened_data)  # Use the defined function
            peak_amp = np.ptp(flattened_data)

            # Hjorth Parameters
            activity, mobility, complexity = hjorth_parameters(flattened_data)

            # Additional Features
            num_peaks = len(find_peaks(flattened_data)[0])
            peak_to_peak_distance = np.mean(np.diff(find_peaks(flattened_data)[0])) if num_peaks > 1 else 0

            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt, zcr, rms_val, energy,
                 crest_fact, shape_fact, signal_entropy_val,
                peak_amp, num_peaks, peak_to_peak_distance,
                activity, mobility, complexity
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)

## Frequency Domain Features


1. Power Spectral Density (PSD) Calculation:

Utilizes the welch method to compute the power spectral density of the signal, which forms the basis for many of the following features.

2. Basic Statistical Features from PSD:

  Mean (mean_val): The average power in the spectrum.

*   Median (median_val): The middle value of the power spectrum.
*   Variance (var_val): The variance of the power spectrum.
*   Standard Deviation (std_dev): The standard deviation of the power spectrum.
*   Standard Deviation (std_dev): The standard deviation of the power spectrum.
* Skewness (skewness): A measure of the asymmetry of the power spectrum.
* Kurtosis (kurt): A measure of the tailedness of the power spectrum.

.

3. Wavelet Coefficients:

* Wavelet Coefficients (wave_coeffs): Computed using the Discrete Wavelet
* Transform (DWT) to provide a multi-resolution analysis of the signal.

* Mean of Wavelet Coefficients (wave_coeffs_mean): The mean of the wavelet coefficients, providing a summary of the wavelet-transformed data.

4. Band Power Features:

Computed by summing the PSD within specified frequency bands. The bands considered are delta, theta, alpha, beta, gamma, and sigma.


5. Band Power Ratios:

* Theta/Alpha Ratio (theta_alpha_ratio): The ratio of power in the theta band to the power in the alpha band.

* Beta/Alpha Ratio (beta_alpha_ratio): The ratio of power in the beta band to the power in the alpha band.

* (Theta + Alpha)/Beta Ratio (theta_alpha_beta_ratio): The ratio of the sum of power in the theta and alpha bands to the power in the beta band.

* Theta/Beta Ratio (theta_beta_ratio): The ratio of power in the theta band to the power in the beta band.

* (Theta + Alpha)/(Alpha + Beta) Ratio (theta_alpha_beta_alpha_ratio): The ratio of the sum of power in the theta and alpha bands to the sum of power in the alpha and beta bands.

* Gamma/Delta Ratio (gamma_delta_ratio): The ratio of power in the gamma band to the power in the delta band.

* (Gamma + Beta)/(Delta + Alpha) Ratio (gamma_beta_delta_alpha_ratio): The ratio of the sum of power in the gamma and beta bands to the sum of power in the delta and alpha bands.

6. Additional Frequency Domain Features:

* Spectral Entropy (spectral_entropy_val): Measures the entropy of the power spectrum, providing an index of the complexity or disorder in the frequency domain.

* Spectral Edge Frequency (spectral_edge_freq): The frequency below which a certain percentage (e.g., 95%) of the power of the signal is contained, providing a cutoff frequency that encloses most of the signal's power.

In [ ]:
# import numpy as np
# from scipy.stats import skew, kurtosis
# from scipy.signal import welch



# def get_wavelet_coeffs(channel_data, wavelet='db4', level=5):
#     coeffs = pywt.wavedec(channel_data, wavelet, level=level)
#     return coeffs


# def extract_frequency_domain_features(epochs, sfreq,wavelet='db4', bands={'delta': (1.59, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'gamma': (30, 100), 'sigma': (11, 16)}):
#     features = []

#     for epoch in epochs:
#         epoch_features = []

#         for channel_data in epoch:
#             # Compute the Power Spectral Density (PSD)
#             freqs, psd = welch(channel_data, sfreq, nperseg=256)

#             # Frequency domain features
#             mean_val = np.mean(psd)
#             median_val = np.median(psd)
#             var_val = np.var(psd)
#             std_dev = np.std(psd)
#             skewness = skew(psd)
#             kurt = kurtosis(psd)

#             # Compute wavelet coefficients
#             wave_coeffs = get_wavelet_coeffs(channel_data, wavelet, level=5)
#             wave_coeffs_mean = np.mean(wave_coeffs[0])

#             # Band Power Features
#             band_powers = {}
#             for band, freq_range in bands.items():
#                 freq_mask = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
#                 band_power = np.sum(psd[freq_mask])
#                 band_powers[band] = band_power

#             # Band Power Ratios
#             theta_alpha_ratio = band_powers['theta'] / band_powers['alpha']
#             beta_alpha_ratio = band_powers['beta'] / band_powers['alpha']
#             theta_alpha_beta_ratio = (band_powers['theta'] + band_powers['alpha']) / band_powers['beta']

#             # Additional Band Power Ratios
#             theta_beta_ratio = band_powers['theta'] / band_powers['beta']
#             theta_alpha_beta_alpha_ratio = (band_powers['theta'] + band_powers['alpha']) / (band_powers['alpha'] + band_powers['beta'])
#             gamma_delta_ratio = band_powers['gamma'] / band_powers['delta']
#             gamma_beta_delta_alpha_ratio = (band_powers['gamma'] + band_powers['beta']) / (band_powers['delta'] + band_powers['alpha'])


#             channel_features = [
#                 mean_val, median_val, var_val, std_dev, skewness, kurt,
#                 band_powers['delta'], band_powers['theta'], band_powers['alpha'],
#                 band_powers['beta'], band_powers['gamma'], band_powers['sigma'],
#                 theta_alpha_ratio, beta_alpha_ratio, theta_alpha_beta_ratio,theta_beta_ratio,
#                 theta_alpha_beta_alpha_ratio, gamma_delta_ratio, gamma_beta_delta_alpha_ratio,
#                 wave_coeffs_mean
#             ]
#             epoch_features.append(channel_features)

#         features.append(epoch_features)

#     return np.array(features)

In [ ]:
import numpy as np
import pywt
from scipy.stats import skew, kurtosis, entropy
from scipy.signal import welch

def get_wavelet_coeffs(channel_data, wavelet='db4', level=5):
    coeffs = pywt.wavedec(channel_data, wavelet, level=level)
    return coeffs


def safe_divide(numerator, denominator, default_value=0.0):
    """Safely divide two numbers, returning a default value if the denominator is zero."""
    if denominator == 0:
        return default_value
    else:
        return numerator / denominator



def spectral_entropy(psd, sfreq):
    # Adding a small constant to avoid division by zero when normalizing
    psd_sum = np.sum(psd) + np.finfo(float).eps
    psd_norm = psd / psd_sum
    # Normalizing PSD to a probability distribution
    psd_norm = psd_norm / np.sum(psd_norm)
    # Ensuring the normalized PSD values are positive
    psd_norm[psd_norm <= 0] = np.finfo(float).eps
    return entropy(psd_norm)


def spectral_edge_frequency(freqs, psd, edge_percent=0.95):
    psd_sum = np.sum(psd) + np.finfo(float).eps  # Add a small constant to ensure non-zero denominator
    psd_cumsum = np.cumsum(psd) / psd_sum
    idx = np.where(psd_cumsum <= edge_percent)[0][-1] if len(psd_cumsum) > 0 else 0
    spectral_edge_freq = freqs[idx] if idx < len(freqs) else 0
    return spectral_edge_freq

def extract_frequency_domain_features(epochs, sfreq, wavelet='db4', bands={'delta': (1.59, 4), 'theta': (4, 8), 'alpha': (8, 12), 'beta': (12, 30), 'sigma': (11, 16)}):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Compute the Power Spectral Density (PSD)
            freqs, psd = welch(channel_data, sfreq, nperseg=256)

            # Frequency domain features
            mean_val = np.mean(psd)
            median_val = np.median(psd)
            var_val = np.var(psd)
            std_dev = np.std(psd)
            skewness = skew(psd)
            kurt = kurtosis(psd)

            # Compute wavelet coefficients
            wave_coeffs = get_wavelet_coeffs(channel_data, wavelet, level=5)
            wave_coeffs_mean = np.mean(wave_coeffs[0])

            # Band Power Features
            band_powers = {}
            for band, freq_range in bands.items():
                freq_mask = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
                band_power = np.sum(psd[freq_mask])
                band_powers[band] = band_power

            # Band Power Ratios
            # Band Power Ratios with safe division
            theta_alpha_ratio = safe_divide(band_powers['theta'], band_powers['alpha'])
            beta_alpha_ratio = safe_divide(band_powers['beta'], band_powers['alpha'])
            theta_alpha_beta_ratio = safe_divide(band_powers['theta'] + band_powers['alpha'], band_powers['beta'])
            theta_beta_ratio = safe_divide(band_powers['theta'], band_powers['beta'])
            theta_alpha_beta_alpha_ratio = safe_divide(band_powers['theta'] + band_powers['alpha'], band_powers['alpha'] + band_powers['beta'])

            # gamma_delta_ratio = band_powers['gamma'] / band_powers['delta']
            # gamma_beta_delta_alpha_ratio = (band_powers['gamma'] + band_powers['beta']) / (band_powers['delta'] + band_powers['alpha'])

            # Additional Features
            spectral_entropy_val = spectral_entropy(psd, sfreq)
            spectral_edge_freq = spectral_edge_frequency(freqs, psd)

            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt,
                band_powers['delta'], band_powers['theta'], band_powers['alpha'],
                band_powers['beta'], band_powers['sigma'],
                theta_alpha_ratio, beta_alpha_ratio, theta_alpha_beta_ratio, theta_beta_ratio,
                theta_alpha_beta_alpha_ratio,
                wave_coeffs_mean, spectral_entropy_val, spectral_edge_freq
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)

#Apply Feature Extraction Methods

In [ ]:
X_time = extract_time_domain_features(X)
sfreq = 256  # Replace with the sampling frequency of your data
# epochs_data = [epoch.get_data() for epoch in epochs]  # Assuming epochs is a list of MNE Epochs objects
X_frequency = extract_frequency_domain_features(X, sfreq)

/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:2017: UserWarning: nperseg = 256 is greater than input length  = 180, using nperseg = 180
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.10/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 5 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [ ]:
#merge the time and frequency features
X_merged_features = np.concatenate((X_time ,X_frequency), axis=2)

X_reshape = X_merged_features.reshape(X_merged_features.shape[0], -1)
# X_reshape = DWT_data.reshape(DWT_data.shape[0], -1)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold, GridSearchCV


import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier,HistGradientBoostingClassifier,\
                                StackingClassifier,VotingClassifier,IsolationForest,\
                                RandomForestRegressor
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import CategoricalNB
from xgboost import XGBClassifier
# import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier

from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV,GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, matthews_corrcoef
from sklearn.model_selection import KFold
import pandas as pd


#Scale the data
SS1= StandardScaler()
X_scaled = SS1.fit_transform(X_reshape)


models = {
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'HistGradientBoosting': HistGradientBoostingClassifier(),
    'SVM': SVC(),
    'GradientBoosting': GradientBoostingClassifier(),
    'KNeighbors': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    # 'MultinomialNB': MultinomialNB(),
    'GaussianNB': GaussianNB(),
    'BernoulliNB': BernoulliNB(),
    'LogisticRegression': LogisticRegression(),
    'AdaBoost': AdaBoostClassifier(),
    # 'Bagging': BaggingClassifier(),
    'ExtraTrees': ExtraTreesClassifier(),

}

def evaluate_model(model, X, y, n_splits):
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=42)
    accuracies, precisions, recalls,f1s,cm,specificitys, sensitivitys,roc_aucs,kappas,mccs= [], [], [],[],[],[],[],[],[],[]

    for train_idx, test_idx in kf.split(X):
      # take the location of the splited data and access it by the index

      X_train, y_train = (X.iloc[train_idx], y.iloc[train_idx]) if isinstance(X, pd.DataFrame) else (X[train_idx], y[train_idx])
      X_test, y_test = (X.iloc[test_idx], y.iloc[test_idx]) if isinstance(X, pd.DataFrame) else (X[test_idx], y[test_idx])
      # train the model
      model.fit(X_train, y_train)
      # make prediction
      y_pred = model.predict(X_test)

      # evaluation scores
      accuracy = accuracy_score(y_test, y_pred)
      precision = precision_score(y_test, y_pred,zero_division=1)
      recall = recall_score(y_test, y_pred)
      f1= f1_score(y_test, y_pred)
      cms = confusion_matrix(y_test, y_pred)  # Calculate confusion matrix
      roc_auc=roc_auc_score(y_test, y_pred)

      # calculate the specificity and sensitivity
      tn, fp, fn, tp = cms.ravel()
      specificity = tn / (tn + fp)
      sensitivity = tp / (tp + fn)
      # Calculate Cohen's Kappa
      kappa = cohen_kappa_score(y_test, y_pred)
      mcc = matthews_corrcoef(y_test, y_pred)





      # append all the results
      accuracies.append(accuracy)
      precisions.append(precision)
      recalls.append(recall)
      f1s.append(f1)
      cm.append(cms)  # Add confusion matrix to list
      specificitys.append(specificity) #add specificity score
      sensitivitys.append(sensitivity) #add sensitivity scores to the list
      roc_aucs.append(roc_auc) #add roc_auc score to the list
      kappas.append(kappa)
      mccs.append(mcc)


    return (format(sum(accuracies)/n_splits, '.3f'), format(sum(precisions)/n_splits, '.3f'),
            format(sum(recalls)/n_splits, '.3f'),format(sum(f1s)/n_splits, '.3f'),format(sum(roc_aucs)/n_splits, '.3f'), sum(cm)/n_splits ,format(sum(specificitys)/n_splits, '.3f'),
            format(sum(sensitivitys)/n_splits, '.3f'),format(sum(kappas)/n_splits, '.3f'),format(sum(mccs)/n_splits, '.3f'))




def modelApplied(Model,X_train , y_train, n_splits=5,MN='model_name',model_result = pd.DataFrame()):
  accuracy, precision, recall,f1,roc_auc,cm,specificity,sensitivity,kappa,mcc = evaluate_model(Model,X_train , y_train, n_splits)

  new_row=[MN,accuracy,precision,recall,f1,roc_auc,specificity, sensitivity,kappa,mcc]

  model_result.loc[len(model_result)]=new_row
  print(new_row)
  print('DONE!!')

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# ['XGBoost', '0.957', '0.987', '0.926', '0.955', '0.957', '0.987', '0.926', '0.913', '0.915']
# DONE!!
# ['RandomForest', '0.937', '0.973', '0.898', '0.934', '0.937', '0.975', '0.898', '0.873', '0.876']
# DONE!!
# ['HistGradientBoosting', '0.958', '0.991', '0.925', '0.957', '0.958', '0.991', '0.925', '0.916', '0.919']
# DONE!!
# ['SVM', '0.916', '0.960', '0.868', '0.911', '0.916', '0.963', '0.868', '0.831', '0.835']
# DONE!!
# ['GradientBoosting', '0.942', '0.975', '0.907', '0.940', '0.942', '0.977', '0.907', '0.884', '0.886']
# DONE!!
# ['KNeighbors', '0.818', '0.988', '0.644', '0.778', '0.818', '0.992', '0.644', '0.636', '0.679']
# DONE!!
# ['MLP', '0.928', '0.953', '0.901', '0.926', '0.928', '0.955', '0.901', '0.856', '0.857']
# DONE!!
# ['DecisionTree', '0.891', '0.887', '0.897', '0.891', '0.891', '0.885', '0.897', '0.782', '0.782']
# DONE!!
# ['GaussianNB', '0.857', '0.985', '0.726', '0.836', '0.857', '0.989', '0.726', '0.714', '0.741']
# DONE!!
# ['BernoulliNB', '0.865', '0.933', '0.787', '0.854', '0.865', '0.943', '0.787', '0.731', '0.740']
# DONE!!
# ['LogisticRegression', '0.899', '0.911', '0.885', '0.897', '0.899', '0.913', '0.885', '0.798', '0.798']
# DONE!!
# ['AdaBoost', '0.923', '0.947', '0.897', '0.921', '0.923', '0.950', '0.897', '0.847', '0.848']
# DONE!!
# ['ExtraTrees', '0.929', '0.964', '0.892', '0.926', '0.929', '0.967', '0.892', '0.858', '0.861']
# DONE!!

#Optimization

###XGB

In [ ]:
import xgboost as xgb
# Initialize XGBClassifier
xgb_model = xgb.XGBClassifier(n_estimators=500, learning_rate=.1, max_depth=5,base_score = 0.5)# default


all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

modelApplied(xgb_model, X_scaled, Y, MN='XGB',model_result= all_band_result)

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# ['XGB', '0.961', '0.991', '0.931', '0.960', '0.961', '0.991', '0.931', '0.923', '0.924']
# DONE!!

###RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Initialize XGBClassifier
RF_model = RandomForestClassifier(n_estimators=150,criterion = 'gini',max_features= 'sqrt')
all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")


modelApplied(RF_model, X_scaled, Y, MN='RF',model_result= all_band_result)

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# ['RF', '0.938', '0.976', '0.899', '0.936', '0.938', '0.978', '0.899', '0.877', '0.880']
# DONE!!

# 1200

###HGB

In [ ]:
pip install bayesian-optimization

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from bayes_opt import BayesianOptimization

X = X_scaled
y = Y

# Define the HistGradientBoostingClassifier and the objective function
def hist_gradient_boosting_cv(learning_rate, max_iter, max_leaf_nodes, max_depth,
                              min_samples_leaf, l2_regularization):
    model = HistGradientBoostingClassifier(
        loss='log_loss',
        learning_rate=learning_rate,
        max_iter=int(max_iter),
        max_leaf_nodes=int(max_leaf_nodes),
        max_depth=int(max_depth),
        min_samples_leaf=int(min_samples_leaf),
        l2_regularization=l2_regularization,
        # max_bins=int(max_bins),
        random_state=42
    )
    # Use cross_val_score for evaluation
    return cross_val_score(model, X, y, scoring='accuracy', cv=5).mean()

# Define the search space for hyperparameters
param_bounds = {
    'learning_rate': (0.01, 1.0),
    'max_iter': (10, 200),
    'max_leaf_nodes': (10, 100),
    'max_depth': (3, 15),
    'min_samples_leaf': (1, 50),
    'l2_regularization': (0.0, 1.0),
    # 'max_bins': (2, 255),
}

# Create BayesianOptimization object
hgbo_optimizer = BayesianOptimization(
    f=hist_gradient_boosting_cv,
    pbounds=param_bounds,
    random_state=42,
)

# Perform optimization
hgbo_optimizer.maximize(
    init_points=5,    # Number of random initializations
    n_iter=10,        # Number of optimization steps
)

# Get the best hyperparameters
best_params = hgbo_optimizer.max['params']
print("Best Hyperparameters:", best_params)

# Train the HistGradientBoostingClassifier with the best hyperparameters
best_hgbo_classifier = HistGradientBoostingClassifier(
    loss='log_loss',
    learning_rate=best_params['learning_rate'],
    max_iter=int(best_params['max_iter']),
    max_leaf_nodes=int(best_params['max_leaf_nodes']),
    max_depth=int(best_params['max_depth']),
    min_samples_leaf=int(best_params['min_samples_leaf']),
    l2_regularization=best_params['l2_regularization'],
    # max_bins=max_bins.astype(int),
    random_state=42
)

# Fit the model on the entire dataset
best_hgbo_classifier.fit(X, y)


|   iter    |  target   | l2_reg... | learni... | max_depth | max_iter  | max_le... | min_sa... |
-------------------------------------------------------------------------------------------------
| 1         | 0.8653    | 0.3745    | 0.9512    | 11.78     | 123.7     | 24.04     | 8.644     |
| 2         | 0.866     | 0.05808   | 0.8675    | 10.21     | 144.5     | 11.85     | 48.53     |
| 3         | 0.8757    | 0.8324    | 0.2202    | 5.182     | 44.85     | 37.38     | 26.71     |
| 4         | 0.8822    | 0.4319    | 0.2983    | 10.34     | 36.5      | 36.29     | 18.95     |
| 5         | 0.8608    | 0.4561    | 0.7873    | 5.396     | 107.7     | 63.32     | 3.276     |
| 6         | 0.8774    | 0.05763   | 0.2478    | 15.0      | 15.87     | 34.77     | 16.67     |
| 7         | 0.8672    | 0.7457    | 0.7344    | 10.15     | 34.79     | 37.36     | 20.88     |
| 8         | 0.8627    | 0.2104    | 0.9926    | 5.567     | 143.2     | 54.61     | 15.1      |
| 9         | 0.8721

HistGradientBoostingClassifier(l2_regularization=0.43194501864211576,
                               learning_rate=0.2983168487960615, max_depth=10,
                               max_iter=36, max_leaf_nodes=36,
                               min_samples_leaf=18, random_state=42)

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
# Initialize XGBClassifier
HGB_model = HistGradientBoostingClassifier( max_leaf_nodes=50,max_iter=300,
                                           max_depth=17, learning_rate = 0.1,
                                            )#max_depth,min_samples_leaf,l2_regularization,max_features

# HGB_model = HistGradientBoostingClassifier(l2_regularization=0.43194501864211576,
#                                learning_rate=0.1, max_depth=17,
#                                max_iter=300, max_leaf_nodes=50,
#                                min_samples_leaf=12, random_state=42)


all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])


print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
modelApplied(HGB_model, X_scaled, Y, MN='HGB',model_result= all_band_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['HGB', '0.962', '0.993', '0.930', '0.960', '0.962', '0.993', '0.930', '0.923', '0.925']
DONE!!


In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# ['HGB', '0.962', '0.993', '0.930', '0.960', '0.962', '0.993', '0.930', '0.923', '0.925']
# DONE!!
# HistGradientBoostingClassifier( max_leaf_nodes=50,max_iter=300,max_depth=17, learning_rate = 0.1)

###GBC

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Initialize XGBClassifier
HGB_model = GradientBoostingClassifier(learning_rate= .01,n_estimators=50)#max_leaf_nodes,max_depth,min_samples_leaf,max_features
all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ALL band <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

modelApplied(HGB_model, X_scaled, Y, MN='GBC',model_result= all_band_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ALL band <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


In [ ]:
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> ALL band <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['GBC', '0.939', '0.969', '0.906', '0.936', '0.939', '0.971', '0.906', '0.877', '0.879']
DONE!!

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
# ['GBC', '0.954', '0.982', '0.925', '0.952', '0.954', '0.982', '0.925', '0.907', '0.909']
# DONE!!
# 450

400

###Extratree

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
# Initialize XGBClassifier
HGB_model = ExtraTreesClassifier(n_estimators=150)#max_leaf_nodes,max_depth,min_samples_leaf,max_features
all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

modelApplied(HGB_model, X_scaled, Y, MN='ETC',model_result= all_band_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming you have your data in X (features) and y (target variable)

# Split the data into training and testing sets with a test size of 20% (0.2)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)


In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score

# Assuming you have your data in X (features) and y (target variable)

# Define the model and parameter grid
model = ExtraTreesClassifier()  # Replace with the classifier you want to optimize
param_grid = {
    'n_estimators': np.arange(50,200,10),  # Number of trees in the forest
    'max_depth': np.arange(15,25),  # Maximum depth of individual trees
    'min_samples_split': [2, 5,7, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 5,7, 10],  # Minimum number of samples required at each leaf node
    'max_features': ['auto', 'log2',None],
    'criterion': ['gini', 'entropy ']# Number of features to consider at each split
}

# Create a GridSearchCV object
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')  # 5-fold CV

# Fit the grid search to the data
grid_search.fit(X_train, Y_train)

# Get the best model and its parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions using the best model
y_pred = best_model.predict(X_test)

# Evaluate the model's performance (replace with other metrics as needed)
accuracy = accuracy_score(Y_test, y_pred)
print("Best Model Parameters:", best_params)
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

###mlp

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
# Initialize XGBClassifier
HGB_model = ExtraTreesClassifier(criterion = 'gini',max_depth=24, max_features=None, min_samples_leaf=5,
                     min_samples_split=10, n_estimators=150, random_state=42)#max_leaf_nodes,max_depth,min_samples_leaf,max_features
all_band_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])

print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

modelApplied(HGB_model, X_scaled, Y, MN='ETC',model_result= all_band_result)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>ALL band<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
['ETC', '0.942', '0.978', '0.905', '0.940', '0.942', '0.979', '0.905', '0.884', '0.887']
DONE!!


#Final Model Exicuition

In [ ]:
X_time.shape, X_frequency.shape, X_merged_features.shape #3 different data is available for analysis

((5834, 29, 18), (5834, 29, 19), (5834, 29, 37))

##Model run without data normalization

###time domain data

data : X_time, X_frequecy, X_merged_features

In [ ]:
from sklearn.model_selection import train_test_split
X_time = X_time.reshape(X_time.shape[0],-1)
X_train, X_test, Y_train,Y_test = train_test_split(X_time,Y, random_state= 42, shuffle = True,test_size=0.20)
X_train.shape, X_test.shape, Y_train.shape,Y_test.shape

((4667, 522), (1167, 522), (4667,), (1167,))

In [ ]:
pd.DataFrame(Y_test).value_counts()

1    587
0    580
dtype: int64

In [ ]:
# time_domain_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])
# for k,v in models.items():
#   modelApplied(v,X_time , Y, n_splits=5,MN=k,model_result = time_domain_result)

['XGBoost', '0.942', '0.980', '0.903', '0.940', '0.942', '0.981', '0.903', '0.884', '0.887']
DONE!!
['RandomForest', '0.935', '0.974', '0.895', '0.933', '0.935', '0.976', '0.895', '0.871', '0.874']
DONE!!
['HistGradientBoosting', '0.943', '0.986', '0.899', '0.940', '0.943', '0.987', '0.899', '0.886', '0.889']
DONE!!
['SVM', '0.791', '0.864', '0.691', '0.768', '0.791', '0.891', '0.691', '0.582', '0.594']
DONE!!
['GradientBoosting', '0.926', '0.962', '0.888', '0.923', '0.926', '0.964', '0.888', '0.853', '0.855']
DONE!!
['KNeighbors', '0.775', '0.825', '0.700', '0.756', '0.775', '0.851', '0.700', '0.551', '0.558']
DONE!!


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


['MLP', '0.832', '0.843', '0.825', '0.831', '0.831', '0.837', '0.825', '0.663', '0.669']
DONE!!
['DecisionTree', '0.889', '0.888', '0.889', '0.889', '0.889', '0.888', '0.889', '0.777', '0.777']
DONE!!
['GaussianNB', '0.771', '0.877', '0.630', '0.733', '0.771', '0.912', '0.630', '0.542', '0.565']
DONE!!
['BernoulliNB', '0.578', '0.577', '0.581', '0.579', '0.578', '0.575', '0.581', '0.156', '0.156']
DONE!!


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

['LogisticRegression', '0.811', '0.828', '0.787', '0.806', '0.811', '0.836', '0.787', '0.623', '0.624']
DONE!!
['AdaBoost', '0.914', '0.941', '0.883', '0.911', '0.914', '0.944', '0.883', '0.828', '0.829']
DONE!!
['ExtraTrees', '0.932', '0.968', '0.892', '0.929', '0.932', '0.971', '0.892', '0.863', '0.866']
DONE!!


In [ ]:
Classifier = {
    'XGBoost': XGBClassifier(n_estimators=170, learning_rate=.2, max_depth=5,base_score = 0.5), # this one is optimized
    'RandomForest': RandomForestClassifier(n_estimators=150,criterion = 'entropy',max_features= None,),# this one is optimized
    'HistGradientBoosting': HistGradientBoostingClassifier( max_leaf_nodes=50,max_iter=300,
                                           max_depth=17, learning_rate = 0.1,
                                            ),# this one is optimized
    # 'SVM': SVC(),
    'GradientBoosting': GradientBoostingClassifier(learning_rate= .2,n_estimators=450,max_depth = 5),
    # 'KNeighbors': KNeighborsClassifier(),
    # 'MLP': MLPClassifier(),
    # 'DecisionTree': DecisionTreeClassifier(),
    # 'MultinomialNB': MultinomialNB(),
    # 'GaussianNB': GaussianNB(),
    # 'BernoulliNB': BernoulliNB(),
    # 'LogisticRegression': LogisticRegression(),
    'AdaBoost': AdaBoostClassifier(estimator = RandomForestClassifier(n_estimators=150,criterion = 'entropy',max_features= None,)),
    # 'Bagging': BaggingClassifier(),
    'ExtraTrees': ExtraTreesClassifier(criterion = 'gini',max_depth=24, max_features=None, min_samples_leaf=5,
                     min_samples_split=10, n_estimators=150, random_state=42),# this one is optimized

}

In [ ]:


time_domain_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])
for k,v in Classifier.items():
  modelApplied(v,X_time , Y, n_splits=5,MN=k,model_result = time_domain_result)

time_domain_result.to_csv('/content/drive/MyDrive/EEG Signal /results/time_domain_features_result.csv')

In [ ]:
X_frequency = X_frequency.reshape(X_frequency.shape[0],-1)
frequency_domain_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])
for k,v in Classifier.items():
  modelApplied(v,X_frequency , Y, n_splits=5,MN=k,model_result = frequency_domain_result)
frequency_domain_result.to_csv('/content/drive/MyDrive/EEG Signal /results/frequency_domain_features_result.csv')

['XGBoost', '0.949', '0.976', '0.920', '0.947', '0.949', '0.977', '0.920', '0.897', '0.899']
DONE!!
['RandomForest', '0.877', '0.894', '0.855', '0.874', '0.877', '0.899', '0.855', '0.753', '0.754']
DONE!!
['HistGradientBoosting', '0.951', '0.979', '0.923', '0.950', '0.951', '0.980', '0.923', '0.903', '0.904']
DONE!!
['GradientBoosting', '0.892', '0.919', '0.859', '0.888', '0.892', '0.924', '0.859', '0.783', '0.785']
DONE!!
['AdaBoost', '0.877', '0.897', '0.852', '0.874', '0.877', '0.902', '0.852', '0.754', '0.755']
DONE!!
['ExtraTrees', '0.882', '0.896', '0.863', '0.879', '0.882', '0.900', '0.863', '0.763', '0.764']
DONE!!


In [ ]:
X_merged_features = X_merged_features.reshape(X_merged_features.shape[0],-1)
combined_domain_result = pd.DataFrame(columns=['Models', 'Accuracy', 'Precision', 'Recall', 'F1-Score', 'Roc_Auc', 'Specificity', 'Sensitivity', 'Kappa', 'MCC'])
for k,v in Classifier.items():
  modelApplied(v,X_scaled , Y, n_splits=5,MN=k,model_result = combined_domain_result)
combined_domain_result.to_csv('/content/drive/MyDrive/EEG Signal /results/combined_domain_features_result.csv')

['XGBoost', '0.961', '0.989', '0.933', '0.960', '0.961', '0.989', '0.933', '0.922', '0.924']
DONE!!
['RandomForest', '0.948', '0.987', '0.907', '0.945', '0.948', '0.988', '0.907', '0.895', '0.899']
DONE!!
['HistGradientBoosting', '0.962', '0.993', '0.930', '0.960', '0.962', '0.993', '0.930', '0.923', '0.925']
DONE!!
['GradientBoosting', '0.960', '0.991', '0.930', '0.959', '0.960', '0.991', '0.930', '0.921', '0.923']
DONE!!
['AdaBoost', '0.945', '0.984', '0.904', '0.942', '0.945', '0.986', '0.904', '0.890', '0.893']
DONE!!
['ExtraTrees', '0.942', '0.978', '0.905', '0.940', '0.942', '0.979', '0.905', '0.884', '0.887']
DONE!!
